In [6]:
# === IMPORT LIBRARIES ===
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import joblib
from skimage.feature import hog
from skimage.transform import resize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import time

# === STEP 1: LOAD DATA ===
lfw_people = fetch_lfw_people(min_faces_per_person=50, resize=0.5)
X_images = lfw_people.images
y = lfw_people.target
target_names = lfw_people.target_names
num_classes = len(np.unique(y))

# === Encode labels ===
le = LabelEncoder()
le.fit(target_names) 
y_enc = le.fit_transform(target_names[y])

# === STEP 2: EXTRACT HOG FEATURES ===
hog_features = []

print("Extracting HOG features...")
start_time = time.time()
for image in tqdm(X_images):
    resized_img = resize(image, (64, 64), anti_aliasing=True)
    features = hog(resized_img,
                   orientations=9,
                   pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2),
                   block_norm='L2-Hys',
                   visualize=False)
    hog_features.append(features)
X_hog = np.array(hog_features)
print("HOG feature shape:", X_hog.shape)
print("Feature extraction time: %.2f seconds" % (time.time() - start_time))

# === STEP 3: PREPARE LABELS ===
y_cat = to_categorical(y_enc, num_classes)

# === STEP 4: SPLIT DATA ===
X_train, X_test, y_train, y_test = train_test_split(X_hog, y_cat, test_size=0.2, random_state=42)

# === STEP 5: DEFINE MLP MODEL (CNN-Like for Vectors) ===
model = Sequential([
    Dense(512, input_shape=(X_train.shape[1],), activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# === STEP 6: TRAIN ===
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=20, batch_size=32, verbose=1)

# === STEP 7: EVALUATE ===
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=target_names))

# === STEP 8: SAVE MODEL AND LABEL ENCODER ===
joblib.dump(le, 'label_encoder.pkl')
model.save('cnn_hog_model.h5')


Extracting HOG features...


100%|█████████████████████████████████████████████████████████████████████████████| 1560/1560 [00:05<00:00, 309.02it/s]

HOG feature shape: (1560, 1764)
Feature extraction time: 5.06 seconds



C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

...

 Total params: 1,040,140 (3.97 MB)

 Trainable params: 1,039,116 (3.96 MB)

 Non-trainable params: 1,024 (4.00 KB)

Epoch 1/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.2729 - loss: 2.6321 - val_accuracy: 0.5288 - val_loss: 1.7741
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.6904 - loss: 1.0011 - val_accuracy: 0.6122 - val_loss: 1.4847
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7909 - loss: 0.6185 - val_accuracy: 0.6474 - val_loss: 1.2259
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8796 - loss: 0.3768 - val_accuracy: 0.7596 - val_loss: 0.9681
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9070 - loss: 0.3046 - val_accuracy: 0.8109 - val_loss: 0.7800
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9356 - loss: 0.2203 - val_accuracy: 0.8494 - val_loss: 0.5821
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9439 - loss: 0.1896 - val_accuracy: 0.8462 - val_loss: 0.5384
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9571 - loss: 0.1317 - val_accuracy: 0.8654 - v


Classification Report:
                   precision    recall  f1-score   support

     Ariel Sharon       0.80      0.73      0.76        11
     Colin Powell       0.90      0.95      0.92        55
  Donald Rumsfeld       0.74      0.92      0.82        25
    George W Bush       0.91      0.91      0.91       107
Gerhard Schroeder       0.67      0.76      0.71        21
      Hugo Chavez       0.82      0.64      0.72        14
   Jacques Chirac       0.83      0.71      0.77         7
    Jean Chretien       1.00      0.75      0.86        12
    John Ashcroft       0.69      0.75      0.72        12
Junichiro Koizumi       0.80      1.00      0.89         8
  Serena Williams       0.83      0.71      0.77         7
       Tony Blair       1.00      0.82      0.90        33

         accuracy                           0.86       312
        macro avg       0.83      0.80      0.81       312
     weighted avg       0.87      0.86      0.86       312



In [1]:
import cv2
import numpy as np
import joblib
from skimage.feature import hog
from tensorflow.keras.models import load_model

# Load face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load label encoder and CNN model
le = joblib.load('label_encoder.pkl')  # Saved during training
model = load_model('cnn_hog_model.h5')  # Your trained model
threshold = 0.5  # Confidence threshold for unknown

def extract_hog_features(image):
    # Resize to 64x64, convert to grayscale (HOG was trained on grayscale images)
    image = cv2.resize(image, (64, 64))
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Extract HOG features; no channel_axis since grayscale
    hog_features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    return hog_features

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    orig = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        features = extract_hog_features(face)  # Pass BGR image, function converts to gray internally
        features = np.expand_dims(features, axis=0)  # Shape (1, n_features)

        probs = model.predict(features, verbose=0)[0]
        max_prob = np.max(probs)
        pred_label = np.argmax(probs)

        if max_prob < threshold:
            name = "Unknown"
        else:
            name = le.inverse_transform([pred_label])[0]

        cv2.rectangle(orig, (x, y), (x+w, y+h), (0, 255, 0), 2)
        label = f"{name} ({max_prob:.2f})"
        cv2.putText(orig, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow('Webcam Face Recognition - HOG + CNN', orig)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
